<a href="https://colab.research.google.com/github/lokeshsai-m/MTP/blob/main/Finetuning_textsummarization(ll-lstm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import numpy as np
import pandas as pd

In [88]:
data = pd.read_csv('/content/dataset1.csv',encoding = 'unicode_escape')
data.drop_duplicates(keep='first')
# data.info()
# print(data.columns)
df = data[['ï»¿abstract', 'conclusion', 'tip', 'no']]

In [89]:
df = df.rename(columns={'no':'id', 'ï»¿abstract':'document', 'tip':'summary'})
# df = df.dropna()

In [90]:
df['document'] = df['document'].apply(lambda x: x.lower())
df['summary'] = df['summary'].apply(lambda x: x.lower())

In [91]:
!pip install transformers==4.17

In [92]:
!pip install datasets rouge-score nltk -q

In [93]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [94]:
!pip install torch

In [95]:
import torch
import datasets
from datasets import Dataset
from datasets import load_metric

In [96]:
import transformers
print(transformers.__version__)

4.17.0


In [97]:
from transformers import AutoTokenizer

model_checkpoint ='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

pad_on_right = tokenizer.padding_side == "right"

loading file https://huggingface.co/t5-small/resolve/main/spiece.model from cache at /root/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
loading file https://huggingface.co/t5-small/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
loading file https://huggingface.co/t5-small/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/t5-small/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/t5-small/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/edbdb128668f5837a316e446d9d0dd59018a797db29e5bb3652db0e8fbe9cda5.679fa4e712151a8d260bfc3f42ace42f9309fe985622073b432663164029e77e


In [98]:
max_input_length = 1024
max_target_length = 128

In [99]:
df = df.sample(frac=1).reset_index(drop=True)
train = df[:40]
valid = df[41:46]
# test = df[47:]
# train.shape, valid.shape,test.shape

In [100]:
def preprocess_function(examples):
    inputs = ['summarize:' + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [101]:
import torch
import datasets
from datasets import Dataset

train = Dataset.from_pandas(train)
valid = Dataset.from_pandas(valid)

In [102]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_valid = valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [103]:
from transformers import TFAutoModelForSeq2SeqLM
from torch import nn

In [104]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [105]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.d67b370cd9d75f81ad4eb421ee7b8db09e0b6a6c693b8c2b423af5d7bcac6205
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_si

In [107]:
batch_size = 16
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [108]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [109]:
import gc
gc.collect()

507

In [110]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))

[INFO] training using Tesla T4


In [111]:
torch.cuda.empty_cache()

In [112]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [113]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-tips",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=200,
    predict_with_generate=True,
    fp16=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [114]:
import tensorflow as tf

In [115]:
model.additional_lstm = tf.keras.layers.LSTM(units=256, return_sequences=True)

In [116]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [117]:
metric = load_metric("rouge")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [118]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: document, conclusion, id, summary. If document, conclusion, id, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40
  Num Epochs = 200
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 600


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.232118,13.037300,3.483000,11.379400,11.452900,19.000000
2,No log,3.205246,13.037300,3.483000,11.379400,11.452900,19.000000
3,No log,3.171407,13.037300,3.483000,11.379400,11.452900,19.000000
4,No log,3.150672,13.037300,3.483000,11.379400,11.452900,19.000000
5,No log,3.117230,11.506800,3.483000,10.575400,10.722500,19.000000
6,No log,3.088376,11.506800,3.483000,10.575400,10.722500,19.000000
7,No log,3.064506,11.506800,3.483000,10.575400,10.722500,19.000000
8,No log,3.041689,11.506800,3.483000,10.575400,10.722500,19.000000
9,No log,3.021501,11.506800,3.483000,10.575400,10.722500,19.000000
10,No log,3.002599,11.506800,3.483000,10.575400,10.722500,19.000000


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: document, conclusion, id, summary. If document, conclusion, id, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: document, conclusion, id, summary. If document, conclusion, id, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: document, conclusion, id, summary. If document, conclusion, id, summary are not expected by `T5For

TrainOutput(global_step=600, training_loss=1.4818703460693359, metrics={'train_runtime': 523.2414, 'train_samples_per_second': 15.289, 'train_steps_per_second': 1.147, 'total_flos': 2165468823552000.0, 'train_loss': 1.4818703460693359, 'epoch': 200.0})

In [119]:
test = df[44:]

In [120]:
print(test.shape)

(5, 4)


In [121]:
eval_dataset = Dataset.from_pandas(test)

In [122]:
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [123]:
predict_results = trainer.predict(
            eval_dataset,max_length=128, num_beams=3)

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: document, conclusion, id, summary. If document, conclusion, id, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5
  Batch size = 16


In [124]:
metrics = predict_results.metrics

In [125]:
metrics

{'test_loss': 1.822241187095642,
 'test_rouge1': 27.3659,
 'test_rouge2': 10.1895,
 'test_rougeL': 21.6781,
 'test_rougeLsum': 23.2226,
 'test_gen_len': 54.4,
 'test_runtime': 1.5731,
 'test_samples_per_second': 3.178,
 'test_steps_per_second': 0.636}

In [126]:
if args.predict_with_generate:
    predictions = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    predictions = [pred.strip() for pred in predictions]

In [127]:
predictions

['abelmoschus esculentus can prevent and manage diabetes associated nephropathy, glaucoma and cataract. regular inclusion of abelmoschus esculentus in daily diet (3 times in a week) can provide effective protection against diabetes and diabetic induced hyperglycemia.',
 'sprouted seeds and microgreens are often more nutrient-dense than ungerminated seeds or mature vegetables. this review also touches on microgreen market trends due to their short growth cycle, nutrient-dense sprouts and microgreens can be produced with minimal input; without pesticides, they can even be home-grown and harvested as needed.',
 'guar gum is a galactomannan derived from the ground endosperm of cluster beans. guar gum is of great interest in regard to weight management and diabetes management.',
 'horse gram is known for its biotic and abiotic resistance, e.g., resistant to drought, salt, and insects.',
 'honey is a novel antidiabetic agent that might be of potential significance for the management of diabe

In [128]:
test['summary']

44    chinese okra is high in antioxidants, which ar...
45    microgreens have enhanced nutritional value du...
46    cluster beans decrease the rate of sugar absor...
47    horse gram has various bioactivities like anti...
48    honey provides better control of the hyperglyc...
Name: summary, dtype: object